# B25 - Training

In [4]:
from gensim.models import Word2Vec
import pickle

In [5]:
with open("../data/tokenized_data/playlist_names/dataset_train_v3.pkl", "rb") as f:
    train_dataset = pickle.load(f)

In [3]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=50, window=5, min_count=1, sg=0
)
word2vec_model.save("../models2/b25-sn-v50/b25-sn-v50.model")

In [4]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=256, window=5, min_count=1, sg=0
)
word2vec_model.save("../models2/b25-sn-v256/b25-sn-v256-a.model")

In [5]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=256, window=10, min_count=1, sg=0
)
word2vec_model.save("../models2/b25-sn-v256/b25-sn-v256-b.model")

In [6]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=256, window=20, min_count=1, sg=0
)
word2vec_model.save("../models2/b25-sn-v256/b25-sn-v256-c.model")

In [3]:
word2vec_model = Word2Vec(
    sentences=train_dataset,
    workers=10,
    vector_size=256,
    window=20,
    min_count=1,
    sg=1,
    ns_exponent=0.0,
    epochs=40,  # prev 5 than 10 than 20
)
word2vec_model.save("../models2/b25-sn-v256/b25-sn-v256-d.model")

In [6]:
word2vec_model = Word2Vec(
    sentences=train_dataset, workers=10, vector_size=512, window=100, min_count=1, sg=0
)
word2vec_model.save("../models2/b25-sn-v512/b25-sn-v512-a.model")

In [6]:
word2vec_model = Word2Vec(
    sentences=train_dataset,
    workers=10,
    vector_size=512,
    window=100,
    min_count=1,
    sg=1,
    ns_exponent=0.0,
    epochs=40,  # c has 10 epochs
)
word2vec_model.save("../models2/b25-sn-v512/b25-sn-v512-b-d.model")

In [3]:
word2vec_model = Word2Vec(
    sentences=train_dataset,
    workers=10,
    vector_size=2048,
    window=100,
    min_count=1,
    sg=1,
    ns_exponent=0.0,
)
word2vec_model.save("../models/b25-sn-v2048/b25-sn-v2048-a.model")

In [ ]:
from algo_testing import song2vec_c

model = song2vec_c.Song2VecC(
    train_dataset, vector_size=512, epochs=15, learning_rate=0.015
)

with open("../models/b25-sn-v512/b25-sn-v512-o.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
from algo_testing import song2vec_c

model = song2vec_c.Song2VecC(
    train_dataset, vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-d.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
# testing with additional score
# the issues is the cluster forming is mostly towards the middle of the box with a varying differentiation of the things
# testing litte teaks but i think there is still the need to pivot a bit more

from algo_testing import song2vec_d

model = song2vec_d.Song2VecD(
    train_dataset, vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-e.pkl", "wb") as f:
    pickle.dump(model, f)
# training was ok took the normal amount. Could use the squard L2 Norm

In [ ]:
from algo_testing import son2vecgl_a

model = son2vecgl_a.Song2GloveA(
    training_data=train_dataset[0:1000], vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-g.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
len(train_dataset)

In [ ]:
from algo_testing import son2vecgl_b

model = son2vecgl_b.Song2GloveB(
    training_data=train_dataset[0:1000], vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-h.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
from algo_testing import son2vecgl_c

model = son2vecgl_c.Song2GloveC(
    training_data=train_dataset, vector_size=512, epochs=15, learning_rate=0.025
)

with open("../models/b25-sn-v512/b25-sn-v512-i.pkl", "wb") as f:
    pickle.dump(model, f)

In [7]:
len(train_dataset)

816957

In [4]:
# cboe
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath("../"))  # Adjust based on the notebook's location

# Now import the `cboe` package
from cboe import *

model = Entity2Vec(
    sentences=train_dataset,
    vector_size=256,
    min_count=1,
    window=100,
    epochs=5,
    learning_rate=0.025,
)

# Save & load
model.save("../models/b25-sn-v256/b25-sn-v256-e.pkl")

Epoch 1/5, Loss: 18405412.4272
Epoch 2/5, Loss: 13423208.3921
Epoch 3/5, Loss: 10481843.0700
Epoch 4/5, Loss: 8869193.0689
Epoch 5/5, Loss: 7794026.0473
Model saved to ../models/b25-sn-v256/b25-sn-v256-e.pkl


In [ ]:
# sge
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath("../"))  # Adjust based on the notebook's location

from sge import *

model = SkipGramEntity2Vec(
    tokenized_data=train_dataset,
    vector_size=256,
    min_count=1,
    epochs=20,
    learning_rate=0.025,
)


model.save("../models/b25-sn-v256/b25-sn-v256-f.pkl")

## Param

#### b25-sn-v50
> vector_size=50, window=5, min_count=1, sg=0

#### b25-sn-v256-a
> vector_size=256, window=5, min_count=1, sg=0

#### b25-sn-v256-b
> vector_size=256, window=10, min_count=1, sg=0

#### b25-sn-v256-c
> vector_size=256, window=20, min_count=1, sg=0

#### b25-sn-v256-d
> vector_size=256, window=20, min_count=1, sg=1, ns_exponent=0.0

#### b25-sn-v512-a
> vector_size=512, window=100, min_count=1, sg=0

#### b25-sn-v512-b
> vector_size=512, window=100, min_count=1, sg=1, ns_exponent=0.0

#### b25-sn-v512-c - CBOS
> vector_size=512, epochs=15, learning_rate=0.015

#### b25-sn-v512-d - CBOS
> vector_size=512, epochs=15, learning_rate=0.025